In [ ]:
# Core Python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
session.sql("USE ROLE TRAINING_ROLE").collect() 
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

In [ ]:
records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()
personnel      = session.table("PERSONNEL_DATA").to_pandas()
worker_comp    = session.table("WORKER_COMP").to_pandas()

In [ ]:
contractors["WORKERS_COMP_COVERAGE_TYPE"].head(100)

In [ ]:
records.columns

In [ ]:
len(records)

In [ ]:
contractors.columns

In [ ]:
#Data clean as we did in class

#Dates
for col in ["ISSUE_DATE", "STATUS_DATE"]:
    records[col] = pd.to_datetime(records[col], errors="coerce")

#Valuation (string -> numeric)
records["VALUATION"] = pd.to_numeric(
        records["VALUATION"].str.replace("$", "").str.replace(",",""),
        errors="coerce")
#Derive CT (LA‑specific tract key) from CENSUS_TRACT if present
ct_numeric = pd.to_numeric(records["CENSUS_TRACT"], errors="coerce")
records["CT"] = (ct_numeric * 100) + 6037000000

In [ ]:
#Clean and Combine datasets
records["LICENSE_NUM"] = records["LICENSE_NUM"].astype(str)
contractors["LICENSE_NO"] = contractors["LICENSE_NO"].astype(str)
contractor_permits = records.merge(contractors,
                                  left_on="LICENSE_NUM",
                                  right_on="LICENSE_NO",
                                  how="inner")
contractor_permits = contractor_permits.rename(columns={"ZIP_CODE_x":"PROJECT_ZIP",
                                                        "ZIP_CODE_y":"COMPANY_ZIP",
                                                       "ISSUE_DATE_x":"PERMIT_DATE",
                                                       "ISSUE_DATE_y":"ESTABLISH_DATE"})
print(len(contractor_permits))


In [ ]:
#1. Who's using local companies(same zip code means local)
contractor_permits["IS_LOCAL"] = (
    contractor_permits["PROJECT_ZIP"].fillna(0) == 
    contractor_permits["COMPANY_ZIP"].fillna(1)
)

#proportion
local_trends = contractor_permits.groupby(contractor_permits["PERMIT_DATE"].dt.year)["IS_LOCAL"].mean()

#plot
plt.figure(figsize=(10, 5))
local_trends.plot(kind="line", marker="o")
plt.title("Percentage of Projects Done by Local Contractors")
plt.ylabel("Share of Local Projects")
plt.grid(True)
plt.show()

In [ ]:
#2. Are top companies gaining most of the revenue?

#Market share by company
total_market_value = contractor_permits["VALUATION"].sum()

top_builders = (
    contractor_permits.groupby("BUSINESS_NAME")["VALUATION"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

top_builders_mkt_share = (top_builders / total_market_value) * 100

#plot 
plt.figure(figsize=(12, 6))
top_builders_mkt_share.sort_values().plot(kind="barh", color="skyblue")
plt.title("Top 10 Contractors by Total Project Valuation")
plt.xlabel("Total Valuation (pct)")
plt.show()

In [ ]:
#3. Do established companies have larger project size

#convert establish date to datetime
contractor_permits["ESTABLISH_DATE"] = pd.to_datetime(contractor_permits["ESTABLISH_DATE"], errors="coerce")

#license age
contractor_permits["LICENSE_AGE"] = (
    contractor_permits["PERMIT_DATE"].dt.year - contractor_permits["ESTABLISH_DATE"].dt.year)

#filter negative and over-60-year values
valid_age = contractor_permits[
    (contractor_permits["LICENSE_AGE"] >= 0) & 
    (contractor_permits["LICENSE_AGE"] <= 60)]


age_valuation = valid_age.groupby("LICENSE_AGE")["VALUATION"].mean()

#plot
plt.figure(figsize=(10, 6))
plt.scatter(age_valuation.index, age_valuation.values, color="teal", alpha=0.6)
#Trend line
z = np.polyfit(age_valuation.index, age_valuation.values, 1)
p = np.poly1d(z)
plt.plot(age_valuation.index, p(age_valuation.index), "r--", alpha=0.8)
plt.title("Average Project Valuation by Contractor Experience")
plt.xlabel("Years Since License Issued")
plt.ylabel("Average Project Valuation ($)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
#4. Regional preference: Rich community vs Normal community

merged_census = contractor_permits.merge(
    census_tracts, 
    left_on="CT", 
    right_on="CENSUS_TRACT", 
    how="inner"
)

#Top 10 busy companies
top_10_names = merged_census["BUSINESS_NAME"].value_counts().head(10).index.tolist()

subset = merged_census[merged_census["BUSINESS_NAME"].isin(top_10_names)]

#Crosstab
cross_tab = pd.crosstab(subset["BUSINESS_NAME"], subset["AMI_CATEGORY"], normalize="index")

ax = cross_tab.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")

plt.title("Top 10 Contractors: Project Distribution by Community Income")
plt.ylabel("Share of Projects")
plt.xlabel("Contractor Name")
plt.legend(title="Income Level", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#5. Compliance: did some of companies really paid for worker injury insurance

#identify the risk level of project
def simple_category(desc):
    d = str(desc).lower()
    if "roof" in d: return "Roofing (High Risk)"
    if "pool" in d: return "Pool (High Risk)"
    if "hvac" in d: return "HVAC"
    if "solar" in d: return "Solar"
    return "Other"

contractor_permits["Project_Type"] = contractor_permits["AI_DESCRIPTION"].apply(simple_category)
plot_data = contractor_permits[contractor_permits["Project_Type"] != "Other"].copy()

#
exempt_rates = plot_data.groupby("Project_Type")["WORKERS_COMP_COVERAGE_TYPE"].apply(
    lambda x: (x == "Exempt").mean() * 100
)

#plot
plt.figure(figsize=(9, 5))
bars = plt.bar(exempt_rates.index, exempt_rates.values, color="salmon")
plt.bar_label(bars, fmt='%.1f%%', padding=3)
plt.title("Percentage of Contractors Claiming 'Exempt' Insurance by Project Type")
plt.ylabel("Percentage Exempt (%)")
plt.ylim(0, 100)
plt.show()